In [ ]:
import math

from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import QFTGate, GlobalPhaseGate, UnitaryGate
from qiskit.circuit import ControlledGate
from qiskit_aer import AerSimulator

from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.quantum_info import random_statevector
import cmath


In [ ]:
# System discretization
n = 3 # number of qubits
length = 5 # full length of x domain

dt = 0.000005

# Potential/system parameters
m = 1.0
omega = 40.0

N = 2**n # x samples (dimension of hilbert space)
bounds = length / 2
dx = length / N

plength = 2 * cmath.pi / dx
dp = plength / N

alpha = dx
beta = (-bounds + dx / 2) / (alpha * n) #see 10.1119/1.2894532
gamma = m * omega**2 * alpha**2 / 2.0
alphap = dp
gammap = alphap**2 / (2.0 * m)


In [ ]:
# Real time evolution e^iV(x) from a harmonic potential
# Returns a function pe(dt) that, when called, returns the PE RTE circuit as a function of dt
# n = number of qubits for state (0 = ancillary, 1 to n will be state qubits)
# for gamma and beta, see DOI 10.1119/1.2894532
# TODO: add support for arbitrary polynomial potentials
def harmpot_rte(n, gamma, beta):
    def pe(dt):
        gate = QuantumCircuit(n)

        phase = GlobalPhaseGate(-gamma * dt * (beta*n)**2)
        gate.append(phase)

        for j in range(n):
            for l in range(n):
                if l == j:
                    gate.p(-gamma * dt * (beta * (2**(l+1)) + 2**(l+j)), l)
                else:
                    gate.p(-gamma * dt * beta * 2**l, l)
                    gate.p(-gamma * dt * beta * 2**j, j)
                    gate.cp(-gamma * dt * 2**(l + j), l, j)
        return gate

    return pe

# Real time evolution e^-idt*T(p) for kinetic energy,
# Returns a function ke(dt) that, when called, returns the KE RTE circuit as a function of dt
# n = number of qubits for state (0 = ancillary, 1 to n will be state qubits)
# see DOI 10.1119/1.2894532
# gammap = dp^2/2m
def kinetic_rte(n, gammap):
    def ke(dt):
        gate = QuantumCircuit(n)

        for j in range(n):
            for l in range(n):
                sign = (-1 if (l == n-1) ^ (j == n-1) else 1)
                exponent = sign * 2**(l+j)
                if l == j:
                    gate.p(-gammap * dt * exponent, l)
                else:
                    gate.cp(-gammap * dt * exponent, l, j)
        return gate

    return ke

In [ ]:
def harm_rte(n, gamma, beta, gammap):
    def rte(dt):
        rte = QuantumCircuit(n)

        # Construct real time evolution circuit
        pot_rte = harmpot_rte(n, gamma, beta)
        ke_rte = kinetic_rte(n, gammap)
        qft = QFTGate(num_qubits=n)
        iqft = qft.inverse()

        rte.append(pot_rte(dt), qargs=range(n))
        rte.append(qft, qargs=range(n))
        rte.append(ke_rte(dt), qargs=range(n))
        rte.append(iqft, qargs=range(n))

        return rte

    return rte

In [ ]:
s2 = math.sqrt(2)
w = Operator([[1/s2, -1j/s2],
             [1/s2, 1j/s2]])
wgate = UnitaryGate(w)

# Probabilistic imaginary time evolution (PITE)
# Returns a function ke(dt) that, when called, returns the ITE circuit as a function of dt (imaginary time)
# rte(dt): A real time evolution wrapper
# m0: real parameter, lower values = more accurate but lower probablilty of succerr
def wrap_ite(n, rte, m0):
    kappa = m0 - (1/math.sqrt(2))
    kappa = kappa/abs(kappa)
    theta0 = kappa * math.acos((m0 + math.sqrt(1 - m0**2))/math.sqrt(2))
    s1 = m0 / math.sqrt(1 - m0**2)

    def ite(dt):
        ite = QuantumCircuit(n+1)
        rte_step = rte(dt*s1)

        ancillary = 0
        args = range(n+1)

        ite.h(ancillary) # H
        ite.append(wgate, [ancillary]) # W
        ite.append(rte_step.control(1, ctrl_state=0), args) # anticontrolled U_RTE
        ite.append(rte_step.inverse().control(1), args) # controlled U_RTE^†
        ite.rz(-2*theta0, ancillary) # Rz(-2theta_0)
        ite.append(wgate.inverse(), [ancillary]) # W^†
        return ite.decompose()


    return ite


In [ ]:
init_state = []
width = 0.3
momentum = 10

def itox(i):
    return -length/2 + (i + 0.5)*dx

for i in range(N):
    init_state.append(0)
    init_state.append(cmath.exp((1j*momentum*itox(i))-((itox(i)/width))**2.0))

init_statevector = Statevector(init_state,
                               tuple([2 for i in range(n+1)]))

In [ ]:
print((counts['00']-counts['10'])/(counts['00']+counts['10']))
print(result.data(0)['statevector'])


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np

def getdata(i):
    if i == 0:
        return result.data(0)['statevector'].data[1::2]
    else:
        return result.data(0)[str(i-1)].data[1::2]


xs = np.linspace((-length+dx)/2,(length-dx)/2,N,endpoint=True)
#ps = np.linspace(0,plength,N,endpoint=False)

fig, ax = plt.subplots()

pspace = ax.plot(xs, getdata(0))[0]
#pspacei = ax.plot(xs, getdata(0)*1j)[0]
#potentialplot = ax.plot(xs, potentials)[0]
ax.set(xlim=[-bounds,bounds])
ax.set(ylim=[-0.5,0.5])

def update(frame):
    data = getdata(frame)
    data_norm = [abs(datum) for datum in data]
    pspace.set_ydata(data_norm)
    #pspacei.set_ydata(getdata(frame)*1j)
    return pspace

ani = animation.FuncAnimation(fig=fig, func=update, frames=50, interval=1)
ani.save('clungus.gif', dpi=80, writer='imagemagick')
plt.show()
